### Data representation
_Вступительный блок о том, как выглядят данные в общем виде_

In [ ]:
# Подключаем необходимые библиотеки
import nibabel as nib
import numpy as np

In [ ]:
# Достаём 3D-изображение из .nii файла
img3d = nib.load("../Dataset/mts-1/FL/Ax Flair mts-1.nii")
type(img3d)

In [ ]:
# Переводим изображение в numpy ndarray
img_data = img3d.get_fdata()
print(f"Object type after get_fdata(): {type(img_data)}")

img_data = np.asarray(img_data)

print(f"\nFinal data type is {type(img_data)} with a shape of (width, height, number of slices, [?]):\n{img_data.shape}")

In [ ]:
# Преобразуем к 3-х мерному массиву
# img_data = img_data.reshape(512, 512, -1)
print(f"Data shape: {img_data.shape}\n")
print(img_data)

In [ ]:
# Достанем картинку (1-ый срез) из полученного массива
from PIL import Image
# img_data_transpose = img_data.T
img = Image.fromarray(img_data[:, 256, :, 0], "L")
img.save("image_1sl.jpeg")

Полученное изображение будет иметь неудовлетворительный для работы вид: 
<img src="image_1sl.jpeg" align="left" alt="bad_image"> <br> 
  <br> 
   *Покрутим ещё*

In [ ]:
# Попробуем лучше строить графики
import matplotlib.pyplot as plt

def show_slices(slices):
    nrows = len(slices) // 5
    fig, axes = plt.subplots(nrows, 5, figsize=(20,20))
    for i in range(nrows):
        for j, slice in enumerate(slices[i*5 : i*5 + 5]):
            axes[i][j].imshow(slice.T, cmap="gray", origin="lower")

In [ ]:
slices = []
for slice_index in range(img_data.shape[2]):
    slices.append(img_data[:, :, slice_index, 0])
show_slices(slices)

### Data preparing
_Непосредственная обработка данных_

In [ ]:
import os.path
from os import listdir

#### Конвертация в 3D-объекты

In [ ]:
def represent_data3D(nii_files):
    nii_objects = []
    for file in nii_files:
        print(file)
        nii_obj = nib.load(file)
        nii_obj = np.asarray(nii_obj.get_fdata())
        
        print(nii_obj.shape)
        
        # Приводим размерность к виду: (channels(slices), width, height)
        nii_obj = nii_obj.T
        
        # Дублируем данные из середины, если это необходимо
        if nii_obj.shape[1] < 20:
            nii_obj = np.concatenate((nii_obj[:, : nii_obj.shape[1]//2, :, :],
                                      nii_obj[:, nii_obj.shape[1]//2 : nii_obj.shape[1]//2 + 2, :, :],
                                      nii_obj[:, nii_obj.shape[1]//2 :, :, :]), axis=1)
        
        # Как бы ни было больно, обрезаем данные 
        if (nii_obj.shape[1] > 40):
            left = (nii_obj.shape[1] - 20) // 3
            right = left  
            midleft = nii_obj.shape[1] // 2 - 4
            midright = nii_obj.shape[1] // 2 + 4
          
            nii_obj = np.concatenate((nii_obj[:, left: left + 6, :, :], \
                                      nii_obj[:, midleft : midright, :, :], \
                                      nii_obj[:, -right-6 : -right, :, :]), axis=1)
        elif(nii_obj.shape[1] > 20):
            if (nii_obj.shape[1] - 20) % 2 == 0:
                left = (nii_obj.shape[1] - 20) // 2
                right = left
            else:
                right = (nii_obj.shape[1] - 20) // 2 + 1
                left = right - 1
            nii_obj = nii_obj[:, left : -right, :, :]
    
        # Ресемплим картинки
        if nii_obj.shape[2:] != (512, 512):
            z1 = 512 / nii_obj.shape[2]
            print(nii_obj.shape)
            z2 = 512 / nii_obj.shape[3]
            nii_obj = ndimage.zoom(nii_obj, (1, 1, z1, z2))
        
        
        # Поворачиваем картинки
        nii_obj = ndimage.rotate(nii_obj, 90, axes=(2,3))
        
        print(nii_obj.shape)
        nii_objects.append(nii_obj)
        
        
    full_nii_data_3D = np.concatenate(tuple(nii_objects), axis=0)
    print(f"Full nii data shape: {full_nii_data_3D.shape}")
    
    if "mts" in file:
        ind_start = file.find("mts")
        ind_fin = file.find("/", ind_start)
        folder_name = "train3D/mts"
        file_name = file[ind_start : ind_fin] + ".nii"
    if "norma" in file:
        ind_start = file.find("norma")
        ind_fin = file.find("/", ind_start)
        folder_name = "train3D/norma"
        file_name = file[ind_start : ind_fin] + ".nii"
    print(folder_name, file_name)
    
    new_nii_file = nib.Nifti1Image(full_nii_data_3D, np.eye(4))
    nib.save(new_nii_file, os.path.join(folder_name, file_name))
    
    print("------------------------------------------------------")

#### Конвертация в 2D-объекты

In [ ]:
def represent_data2D(nii_files):
    nii_objects = []
    for file in nii_files:
        print(file)
        nii_obj = nib.load(file)
        nii_obj = np.asarray(nii_obj.get_fdata())
        
        print(nii_obj.shape)
        
        # Приводим размерность к виду: (channels(slices), width, height)
        nii_obj = nii_obj.reshape(nii_obj.shape[0], nii_obj.shape[1], -1).T
        
        # Дублируем данные из середины, если это необходимо
        if nii_obj.shape[0] < 20:
            nii_obj = np.concatenate((nii_obj[: nii_obj.shape[0]//2, :, :],
                                      nii_obj[nii_obj.shape[0]//2 : nii_obj.shape[0]//2 + 2, :, :],
                                      nii_obj[nii_obj.shape[0]//2 :, :, :]), axis=0)
        
        # Как бы ни было больно, обрезаем данные 
        if (nii_obj.shape[0] > 40):
            left = (nii_obj.shape[0] - 20) // 3
            right = left  
            midleft = nii_obj.shape[0] // 2 - 4
            midright = nii_obj.shape[0] // 2 + 4
          
            nii_obj = np.concatenate((nii_obj[left: left + 6, :, :], \
                                      nii_obj[midleft : midright, :, :], \
                                      nii_obj[-right-6 : -right, :, :]), axis=0)
        elif(nii_obj.shape[0] > 20):
            if (nii_obj.shape[0] - 20) % 2 == 0:
                left = (nii_obj.shape[0] - 20) // 2
                right = left
            else:
                right = (nii_obj.shape[0] - 20) // 2 + 1
                left = right - 1
            nii_obj = nii_obj[left : -right, :, :]
    
        # Ресемплим картинки
        if nii_obj.shape[1:] != (512, 512):
            z1 = 512 / nii_obj.shape[1]
            z2 = 512 / nii_obj.shape[2]
            nii_obj = ndimage.zoom(nii_obj, (1, z1, z2))
        
        
        # Поворачиваем картинки
        nii_obj = ndimage.rotate(nii_obj, 90, axes=(1,2))
        
        print(nii_obj.shape)
        nii_objects.append(nii_obj)
        
        
    full_nii_data_2D = np.concatenate(tuple(nii_objects), axis=0)
    print(f"Full nii data shape: {full_nii_data_2D.shape}")
    
    if "mts" in file:
        ind_start = file.find("mts")
        ind_fin = file.find("/", ind_start)
        folder_name = "mts"
        file_name = file[ind_start : ind_fin] + ".nii"
    if "norma" in file:
        ind_start = file.find("norma")
        ind_fin = file.find("/", ind_start)
        folder_name = "norma"
        file_name = file[ind_start : ind_fin] + ".nii"
    print(folder_name, file_name)
    
    new_nii_file = nib.Nifti1Image(full_nii_data_2D, np.eye(4))
    nib.save(new_nii_file, os.path.join(folder_name, file_name))
    
    print("------------------------------------------------------")

#### Базовый алгоритм

In [ ]:
# Для всех
start_path = "../Dataset"
count = 0
for folder in listdir(start_path):
    nii_files = []
    folder_path = start_path + "/" + folder
    # print(folder_path)
    if os.path.isdir(folder_path):
        
        for subfolder in listdir(folder_path):
            subfolder_path = folder_path + "/" + subfolder
            if os.path.isdir(subfolder_path):
                
                for file in listdir(subfolder_path):
                    file_path = subfolder_path + "/" + file
                    if file.endswith(".nii"):
                        nii_files.append(file_path)

        represent_data3D(nii_files)
        count+=1
        
print(f"Total: {count} patients data")

#### Проверка для 2D

In [ ]:
# Посмотрим на снимки, которые у нас есть по одному пациенту
p1_file = nib.load("train2D/mts/mts-3.nii")
p1 = np.asarray(p1_file.get_fdata())
print(p1.shape)
for k in range(3):
    slices = []
    shots = p1.shape[0] // 3
    for slice_index in range(shots):
        slices.append(p1[k * shots + slice_index, :, :])
    show_slices(slices)

#### Проверка для 3D

In [ ]:
p2_file = nib.load("train3D/norma/norma-1.nii")
p2 = np.asarray(p2_file.get_fdata())
print(p2.shape)
for i in range(p2.shape[0]):
    slices = []
    for slice_index in range(p2.shape[1]):
        slices.append(p2[i, slice_index, :, :])
    show_slices(slices)

## That's all, folks!
<!-- <div align="center" style="font-size: 20px; font-weight: bold">That's all, folks!</div> -->